### DB 연결

In [2]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
import time
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine

In [2]:
#pip freeze > requirements.txt

In [3]:
import psycopg2
from sshtunnel import SSHTunnelForwarder


try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


### 테이블확인

In [ ]:
# 어떤 변수들부터 뽑아볼까?
# ventilator mode
# Peak pressure(cmH2O)
# Plateau pressure(cmH2O)
# Mean pressure(cmH2O)
# PEEP ((cmH2O))
# Tidal volume (mL)
# FiO2 (%)
# Phigh(cmH2O)
# Plow(cmH2O)
# Thigh(sec)
# Tlow(sec)

In [4]:
# curs.execute("select * from mimiciv_derived.ventilator_setting;")
# columns_name = [desc[0] for desc in curs.description]
# ventilator_setting = pd.DataFrame(curs.fetchall(), columns=columns_name) 
# ventilator_setting

,subject_id,stay_id,charttime,respiratory_rate_set,respiratory_rate_total,respiratory_rate_spontaneous,minute_volume,tidal_volume_set,tidal_volume_observed,tidal_volume_spontaneous,plateau_pressure,peep,fio2,flow_rate,ventilator_mode,ventilator_mode_hamilton,ventilator_type
0,10000980,39765666,2189-06-27 09:00:00,NaN,27.0,27.0,11.2,NaN,393.0,528.0,NaN,5.0,NaN,NaN,CPAP/PSV,None,Drager
1,10000980,39765666,2189-06-27 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,Standby,None,None
2,10001884,37510196,2131-01-11 04:00:00,20.0,20.0,0.0,9.0,450.0,479.0,NaN,24.0,5.0,100.0,39.9,None,APV (cmv),Hamilton
3,10001884,37510196,2131-01-11 07:00:00,20.0,20.0,0.0,8.0,400.0,408.0,NaN,22.0,5.1,50.0,35.6,None,APV (cmv),Hamilton
4,10001884,37510196,2131-01-11 11:00:00,20.0,20.0,0.0,8.3,400.0,430.0,NaN,23.0,5.2,50.0,32.3,None,APV (cmv),Hamilton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1006122,19999987,36195440,2145-11-03 22:37:00,NaN,29.0,29.0,8.8,NaN,429.0,641.0,NaN,10.0,50.0,NaN,CPAP/PSV,None,None
1006123,19999987,36195440,2145-11-04 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standby,None,None
1006124,19999987,36195440,2145-11-04 07:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,None,None,None
1006125,19999987,36195440,2145-11-04 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN,None,None,None


In [5]:
#ventilator_setting.to_csv('../outputs/ventilator_setting.csv', index=False, encoding='utf-8-sig')
ventilator_setting = pd.read_csv('../outputs/ventilator_setting.csv', parse_dates=['charttime'])

In [6]:
ventilator_setting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006127 entries, 0 to 1006126
Data columns (total 17 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   subject_id                    1006127 non-null  int64         
 1   stay_id                       1006127 non-null  int64         
 2   charttime                     1006127 non-null  datetime64[ns]
 3   respiratory_rate_set          342289 non-null   float64       
 4   respiratory_rate_total        550398 non-null   float64       
 5   respiratory_rate_spontaneous  592180 non-null   float64       
 6   minute_volume                 607284 non-null   float64       
 7   tidal_volume_set              327557 non-null   float64       
 8   tidal_volume_observed         611925 non-null   float64       
 9   tidal_volume_spontaneous      321855 non-null   float64       
 10  plateau_pressure              217024 non-null   float64       
 11

In [7]:
subjectlist1 = pd.read_csv('./data/subjectlist1.csv', parse_dates=['admittime','intubationtime','dischtime', 'deathtime','extubationtime','reintubation_eventtime'])
subjectlist1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9003 entries, 0 to 9002
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              9003 non-null   int64         
 1   subject_id              9003 non-null   int64         
 2   hadm_id                 9003 non-null   int64         
 3   stay_id                 9003 non-null   int64         
 4   int_stayid              7860 non-null   float64       
 5   admittime               9003 non-null   datetime64[ns]
 6   intubationtime          7860 non-null   datetime64[ns]
 7   int_itemid              7860 non-null   float64       
 8   int_weight              7860 non-null   float64       
 9   ext_stayid              6825 non-null   float64       
 10  extubationtime          8759 non-null   datetime64[ns]
 11  ext_itemid              6825 non-null   float64       
 12  ext_weight              6825 non-null   float64 

In [8]:
# curs.execute("select * from mimiciv_derived.ventilation;")
# columns_name = [desc[0] for desc in curs.description]
# ventilation = pd.DataFrame(curs.fetchall(), columns=columns_name) 
# ventilation

In [30]:
# #ventilation.to_csv('../outputs/ventilation.csv', index=False, encoding='utf-8-sig')
# ventilation = pd.read_csv('../outputs/ventilation.csv', parse_dates=['starttime','endtime'])

In [9]:
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id')

In [11]:
ventilator_setting.head(4)

,subject_id,stay_id,charttime,respiratory_rate_set,respiratory_rate_total,respiratory_rate_spontaneous,minute_volume,tidal_volume_set,tidal_volume_observed,tidal_volume_spontaneous,plateau_pressure,peep,fio2,flow_rate,ventilator_mode,ventilator_mode_hamilton,ventilator_type
0,10000980,39765666,2189-06-27 09:00:00,NaN,27.0,27.0,11.2,NaN,393.0,528.0,NaN,5.0,NaN,NaN,CPAP/PSV,NaN,Drager
1,10000980,39765666,2189-06-27 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,Standby,NaN,NaN
2,10001884,37510196,2131-01-11 04:00:00,20.0,20.0,0.0,9.0,450.0,479.0,NaN,24.0,5.0,100.0,39.9,NaN,APV (cmv),Hamilton
3,10001884,37510196,2131-01-11 07:00:00,20.0,20.0,0.0,8.0,400.0,408.0,NaN,22.0,5.1,50.0,35.6,NaN,APV (cmv),Hamilton


In [13]:
sub1_ventset = sub1list >> left_join(ventilator_setting, by=['subject_id','stay_id'])

In [14]:
sub1_ventset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569429 entries, 0 to 569428
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   subject_id                    569429 non-null  int64         
 1   hadm_id                       569429 non-null  int64         
 2   stay_id                       569429 non-null  int64         
 3   charttime                     569390 non-null  datetime64[ns]
 4   respiratory_rate_set          200586 non-null  float64       
 5   respiratory_rate_total        329939 non-null  float64       
 6   respiratory_rate_spontaneous  363899 non-null  float64       
 7   minute_volume                 378046 non-null  float64       
 8   tidal_volume_set              192266 non-null  float64       
 9   tidal_volume_observed         369043 non-null  float64       
 10  tidal_volume_spontaneous      208347 non-null  float64       
 11  plateau_press

In [15]:
sub1_ventset.to_csv('../outputs/sub1_ventset.csv', index=False, encoding='utf-8-sig')

In [ ]:

# Remember to close the SSH tunnel
tunnel.close()